# Web Scraping: Selenium

Selenium también tiene varios métodos que facilitan la extracción de datos.

En primer lugar, tendremos que descargar un controlador.

Usaremos ChromeDriver para Google Chrome. Para obtener una lista completa de controladores y plataformas compatibles, consulte [Selenium](https://www.selenium.dev/downloads/). Si desea utilizar Google Chrome, diríjase a [chrome](https://chromedriver.chromium.org/) y descargue el controlador que corresponde a su versión actual de Google Chrome.

Como saber cual es la version de chrome que utilizo simple utilizamos pegamos el siguiente enlace en la barra de chrome chrome://settings/help

El proceso de web scraping con Selenium es el siguiente:

URL → Solicitud HTTP → HTML → Selenium → DOM



## Comencemos importando las bibliotecas que usaremos:

In [ ]:
!pip install selenium

In [2]:
from selenium import webdriver
import urllib3 # urllib3 es un cliente HTTP potente y fácil de usar para Python.
import re # Expresiones regulares 
import time
import pandas as pd

El objeto driver es con el que trabajaremos a partir de ahora

In [3]:
# especificamos el path hasta nuestro driver recién descargado:
chrome_driver_path = 'chromedriver.exe'
options  = webdriver.ChromeOptions()

In [4]:
# Creamos el driver con el que nos vamos a manejar en la sesión de scrapeo:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

C:\Users\Alex\AppData\Local\Temp\ipykernel_26988\2445550838.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)


In [5]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://insolvencyinsider.ca/filing/'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)

Ahora si queremos hacer click en el boton de "Load more"..

Selenium proporciona varios métodos para localizar elementos en la página web. Usaremos el método find_element_by_xpath() para crear un objeto de botón, con el que luego podremos interactuar:

/html/body/div[2]/div/main/div/div/div/button

In [6]:
from selenium.webdriver.common.by import By

In [7]:
loadMore = driver.find_element(By.XPATH, '/html/body/div[2]/div/main/div/div/div/button')

In [ ]:
# loadMore = driver.find_element(By.XPATH, '//*[@id="content"]/button')

Antes de continuar, necesitaremos saber cuántas páginas hay para saber cuántas veces debemos hacer clic en el botón.

In [8]:
http = urllib3.PoolManager()
r = http.request("GET", url)
text = str(r.data)

In [ ]:
text

```text``` ahora es una cadena de texto. Gracias a la librería Regex, podemos extraer el número de páginas total.

In [10]:
paginas_totales_obj = re.search(pattern='"total_pages":\d+', string=text)
paginas_totales_str = paginas_totales_obj.group(0)
paginas_totales = int(re.search(pattern="\d+", string=paginas_totales_str).group(0))

In [11]:
paginas_totales

97

In [12]:
from tqdm import tqdm

for i in tqdm(range(paginas_totales-1)):
    loadMore.click()
    time.sleep(3)

100%|██████████| 96/96 [04:52<00:00,  3.04s/it]


Con eso completo, ahora podemos cargar todas las páginas de Insolvency Insider. Podemos hacer clic en el botón Cargar más accediendo al método click() del objeto. Esperamos tres segundos entre clics para no sobrecargar el sitio web.

Recuerde que el total de las páginas son 88 pero comenzamos en 0 asi que es 88-1

Una vez que ejecute esto, debería ver que se hace clic en el botón Cargar más y que se cargan las páginas restantes.
Una vez que se carga cada página, podemos comenzar a raspar el contenido. Ahora, eliminar ciertos elementos como el nombre de presentación, la fecha y la hiperreferencia es bastante sencillo. Podemos usar los métodos find_elements_by_class_name() y find_elements_by_xpath() de Selenium (importante la ```s``` extra después de element):

filing-name
filing-date
//*[@id='content']/div[2]/div/div[1]/h3/a

In [17]:
filing_names_elementos = driver.find_elements(By.CLASS_NAME, 'filing-name')
filing_dates_elementos = driver.find_elements(By.CLASS_NAME, 'filing-date')


In [25]:
filing_href_elementos = driver.find_elements(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/h3/a')

También nos gustaría conocer los metadatos de presentación, es decir, el tipo de archivo, el sector de la empresa y el lugar en la que operan. Extraer estos datos requiere un poco más de trabajo.

//*[@id='content']/div[2]/div[%d]/div[2]/div[1]

In [26]:
filing_href_elementos

[<selenium.webdriver.remote.webelement.WebElement (session="93e7e8620a33a83f48c1e1e98836c50d", element="c1bb588a-8820-4d84-beec-31bd492dc902")>,
 <selenium.webdriver.remote.webelement.WebElement (session="93e7e8620a33a83f48c1e1e98836c50d", element="c87297f7-a9d9-4a98-928f-d39885844b92")>,
 <selenium.webdriver.remote.webelement.WebElement (session="93e7e8620a33a83f48c1e1e98836c50d", element="77e567bd-e816-4fde-ac72-a8fe9570c697")>,
 <selenium.webdriver.remote.webelement.WebElement (session="93e7e8620a33a83f48c1e1e98836c50d", element="03dd812a-4323-421a-98c6-43c555e8e4a8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="93e7e8620a33a83f48c1e1e98836c50d", element="6eb76a2e-b36e-47d9-b9be-c24fba5f7261")>,
 <selenium.webdriver.remote.webelement.WebElement (session="93e7e8620a33a83f48c1e1e98836c50d", element="e5abf7e5-6021-4704-8eea-f087f8e250bb")>,
 <selenium.webdriver.remote.webelement.WebElement (session="93e7e8620a33a83f48c1e1e98836c50d", element="b103e433-b3b0-4a21-8316-30

De cada elemento de la presentación de Metas podemos extraer el tipo de presentación, la industria y la provincia, así:

********

In [18]:
filingName = []
filingDate = []
filingLink = []
# para cada elemento en la lista de elementos de nombre de archivo, agrega el
# texto del elemento a la lista de nombres de archivo.
for element in tqdm(filing_names_elementos):
    filingName.append(element.text)
# para cada elemento en la lista de elementos de la fecha de presentación, agrega el
# texto del elemento a la lista de fechas de presentación.
for element in tqdm(filing_dates_elementos):
    filingDate.append(element.text)


100%|██████████| 1/1 [00:00<00:00, 14.44it/s]


In [27]:
for link in tqdm(filing_href_elementos):
    if link.get_attribute("href"):
        filingLink.append(link.get_attribute("href"))

100%|██████████| 966/966 [00:18<00:00, 53.62it/s]


In [30]:
filingLink.pop(0)

'https://insolvencyinsider.ca/filing/home-solutions-corporation-2/'

In [31]:
len(filingLink)

966

*********

Ahora, todavía tenemos que poner nuestros nombres y fechas de presentación en las listas. Hacemos esto agregando el texto de cada elemento a una lista usando el método text() de antes:

Una vez que tengamos eso, estamos listos para poner todo en un diccionario y luego crear un DataFrame de pandas:

In [32]:
# Crea un diccionario final con nombres y fechas de archivo.

fullDict = {
    "Filing Name": filingName,
    "Filing Date": filingDate, 
    "Link": filingLink
}
# Crea un DataFrame.
df = pd.DataFrame(fullDict)
df["Filing Date"] = pd.to_datetime(df["Filing Date"], infer_datetime_format=True)

In [33]:
df

,Filing Name,Filing Date,Link
0,Home Solutions Corporation,2022-07-11,https://insolvencyinsider.ca/filing/home-solut...
1,2737524 Ontario Inc. o/a Estrada Automotive,2022-07-05,https://insolvencyinsider.ca/filing/2737524-on...
2,Premium Comfort Heating & Air Conditioning Ltd,2022-06-28,https://insolvencyinsider.ca/filing/premium-co...
3,"Sproutly Inc. and its wholly-owned subsidiary,...",2022-06-24,https://insolvencyinsider.ca/filing/sproutly-i...
4,Revlon Canada Inc. and Elizabeth Arden (Canada...,2022-06-20,https://insolvencyinsider.ca/filing/revlon-can...
...,...,...,...
961,ReidBuilt,2017-11-02,https://insolvencyinsider.ca/filing/reidbuilt/
962,Spareparts,2017-10-31,https://insolvencyinsider.ca/filing/spareparts/
963,BuildDirect,2017-10-31,https://insolvencyinsider.ca/filing/builddirect/
964,1735549 Ontario,2017-10-27,https://insolvencyinsider.ca/filing/1735549-on...


------------------------

# Ahora algo más visual

In [34]:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

C:\Users\Alex\AppData\Local\Temp\ipykernel_26988\3395105876.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)


In [35]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://www.filmaffinity.com/es/main.html'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)

In [39]:
elements_by_tag = driver.find_elements(By.TAG_NAME, 'button')
elements_by_class = driver.find_elements(By.CLASS_NAME, 'css-2tkghh')
elements_by_xpath = driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')

Podemos extraer todos los atributos que tenga

In [40]:
dir(elements_by_xpath)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 'accessible_name',
 'aria_role',
 'clear',
 'click',
 'find_element',
 'find_elements',
 'get_attribute',
 'get_dom_attribute',
 'get_property',
 'id',
 'is_displayed',
 'is_enabled',
 'is_selected',
 'location',
 'location_once_scrolled_into_view',
 'parent',
 'rect',
 'screenshot',
 'screenshot_as_base64',
 'screenshot_as_png',
 'send_keys',
 'shadow_root',
 'size',
 'submit',
 'tag_name',
 'text',
 'value_of_css_property']

Podemos evaluar que tipo de elemento es (tag)

In [41]:
elements_by_xpath.tag_name

'button'

Podemos sacar el valor que tiene (el texto)

In [42]:
elements_by_xpath.text

'ACEPTO'

Incluso podemos guardar una imagen del elemento

Evaluamos que elementos hemos encontrado por el tag:

Seguimos

Instanciamos el elemento del tag [2] en la variable boton aceptar

In [43]:
boton_aceptar = elements_by_xpath

Si el elemento es interactivo podremos hacer más cosas además de las anteriores. Por ejemplo: hacer click

In [44]:
boton_aceptar.click()

Buscamos una película por título

In [45]:
from selenium.webdriver.common.keys import Keys

/html/body/div[2]/div[1]/div/div[2]/form/div/input

In [46]:
buscador = driver.find_element(By.XPATH, '//*[@id="top-search-input"]')
buscador.send_keys('El Caballero Oscuro')

In [47]:
buscador.send_keys(Keys.ENTER)

In [48]:
driver.back()

### Vamos a buscar todas las películas que se estrenan el próximo viernes

1. Cogemos los containers que hay en la zona lateral

In [58]:
menu_lateral = driver.find_element(By.ID, 'lsmenu') ####### si se corrompe el navegador seleciona cualquier pagina distinta a proximos extrenos y ejecuta desde aqui
menu_lateral

<selenium.webdriver.remote.webelement.WebElement (session="19fa22ac493b1b6a75a7731da9c2357f", element="950ac831-9bcd-4a92-bd9f-2f5bce481dcc")>

In [59]:
menu_lateral.text

'Usuarios\nVotar los tours\nIniciar sesión\nRegistrarse\nEspaña\nPelículas en cartelera\nCines España\nPróximos estrenos\nEstrenos Blu-ray venta\nPróximos Blu-ray venta\nYa para alquilar\nPróximamente alquiler\nNetflix\nNetflix (próx)\nAmazon Prime\nMovistar Plus+\nMovistar Plus+ (próx)\nHBO Max\nDisney+\nFilmin\nApple TV+\nRakuten TV\nUSA - UK - FR\nEstrenos USA\nEstrenos Reino Unido\nEstrenos Francia\nRankings FA\nTodos los rankings\nN\nEl Top 1000 de FA\nSecciones\nTaquilla\nTrailers\nÚltimos trailers\nÚltimas críticas users\nQué dice la crítica\nPelículas por temas\nSagas & Franquicias\nSeries de TV\nSeries de actualidad\nTop series\nVota Series de TV\nTOPs\nTop Filmaffinity\nLo mejor del TOP\nTop Estrenos\nTop Estrenos Blu-ray\nRanking de listas\nPremios | Festivales\nPremios y Festivales\nTodos los Oscar\nResumen 2021\nInformación\nAcerca de FA\nContacto'

In [60]:
mis_secciones = menu_lateral.find_elements(By.TAG_NAME, 'a')

In [ ]:
mis_secciones

2. Vemos con cuál nos tenemos que quedar

In [52]:
for a in tqdm(mis_secciones):
    if a.text == 'Próximos estrenos':
        a.click()
        break

 11%|█         | 5/45 [00:01<00:08,  4.62it/s]


In [54]:
driver.back()

In [62]:
for a in tqdm(mis_secciones):
    if a.text == 'Próximos estrenos':
        a.click()
        break

 11%|█         | 5/45 [00:00<00:07,  5.49it/s]


Accedemos al container central, en el que aparecen los estrenos por semana que queremos ver, exactamente igual que hemos hecho antes

In [63]:
cajon_central = driver.find_elements(By.ID, 'main-wrapper-rdcat')

In [64]:
for semana in cajon_central:
    print(semana.find_element(By.TAG_NAME, 'div').text)
    print(semana.find_element(By.TAG_NAME, 'div').get_attribute('id'))

14 de julio de 2022
2022-07-14
22 de julio de 2022
2022-07-22
29 de julio de 2022
2022-07-29
5 de agosto de 2022
2022-08-05
12 de agosto de 2022
2022-08-12
19 de agosto de 2022
2022-08-19
26 de agosto de 2022
2022-08-26


In [65]:
caratulas = semana.find_elements(By.CLASS_NAME, 'mc-poster')
lista_pelis = []
for peli in caratulas:
    lista_pelis.append(peli.find_element(By.TAG_NAME, 'a').get_attribute('href'))

In [66]:
lista_pelis

['https://www.filmaffinity.com/es/film708007.html',
 'https://www.filmaffinity.com/es/film868866.html',
 'https://www.filmaffinity.com/es/film840601.html',
 'https://www.filmaffinity.com/es/film259770.html',
 'https://www.filmaffinity.com/es/film157744.html',
 'https://www.filmaffinity.com/es/film227486.html',
 'https://www.filmaffinity.com/es/film479695.html']

Buscamos cómo acceder a las películas

Una vez tenemos todas las urls vamos a ver qué hacemos con cada una de ellas

In [67]:
driver.get(lista_pelis[0])

Vamos a ver el proceso que deberíamos hacer con cada una de las películas:

1. Sacamos toda la información que nos interesa

In [70]:
titulo = driver.find_element(By.XPATH, '//*[@id="main-title"]/span').text
print(titulo)
nota = driver.find_element(By.XPATH, '/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]').text
votos = driver.find_element(By.XPATH, '/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span').text
ficha = driver.find_element(By.XPATH, '/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')

La invitación


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x011D6463+2188387]
	Ordinal0 [0x0116E461+1762401]
	Ordinal0 [0x01083D78+802168]
	Ordinal0 [0x010B1880+989312]
	Ordinal0 [0x010B1B1B+989979]
	Ordinal0 [0x010DE912+1173778]
	Ordinal0 [0x010CC824+1099812]
	Ordinal0 [0x010DCC22+1166370]
	Ordinal0 [0x010CC5F6+1099254]
	Ordinal0 [0x010A6BE0+945120]
	Ordinal0 [0x010A7AD6+948950]
	GetHandleVerifier [0x014771F2+2712546]
	GetHandleVerifier [0x0146886D+2652765]
	GetHandleVerifier [0x0126002A+520730]
	GetHandleVerifier [0x0125EE06+516086]
	Ordinal0 [0x0117468B+1787531]
	Ordinal0 [0x01178E88+1805960]
	Ordinal0 [0x01178F75+1806197]
	Ordinal0 [0x01181DF1+1842673]
	BaseThreadInitThunk [0x76806739+25]
	RtlGetFullPathName_UEx [0x77AD8FEF+1215]
	RtlGetFullPathName_UEx [0x77AD8FBD+1165

2. Creamos una lista a partir de la ficha técnica

In [73]:
titulo_names = []

for name in titulo.find_elements(By.TAG_NAME, 'dt'):
    titulo_names.append(name.text)

AttributeError: 'str' object has no attribute 'find_element'

3. Creamos un dataframe con la info

Ahora vamos a crear una función que nos haga todo esto para cada una de las películas:

Vamos a ver cómo nos podemos mover entre ventanas del navegador

Abrir nueva ventana:

Movernos a otra ventana

Cerrar ventana

Una vez cerramos la ventana tenemos que indicarle a qué ventana tiene que ir

Sabiendo cómo podemos movernos por entre las ventanas y sabiendo cómo extraer de cada página toda la información que necesitamos vamos a crear nuestro dataframe:

Ya tenemos un dataframe con todas las películas que se van a estrenar el próximo viernes